In [1]:
import os
import json
import copy

import sys
sys.path.append("../mpi_scripts/voigt")
from io_utils import strip_comments

In [2]:
def rel_join(start, *args):
    path = os.path.join(*args)
    relpath = os.path.relpath(path, start=start)
    # abspath = os.path.abspath(os.path.relpath(relpath))
    # realpath = os.path.realpath(relpath)
    return os.path.abspath(path) # relpath

# Synthetic

In [3]:
entry_point = "../mpi_scripts/voigt/mpi_script.py"

## top level

In [4]:
dirname_results = "../results/test/"

In [5]:
ga_dict = {"seed": 42,
           "n_organisms": 1024,
           "n_elites": 32,
           "n_generations": 500,
           "gamma": 0.015,
           "loss": "V_CaT_shared",
           "columns_control": ["V", "fluo"],
           "columns_model": ["V", "fluo"],}

In [6]:
# test
ga_dict = {"seed": 42,
           "n_organisms": 32,
           "n_elites": 2,
           "n_generations": 10,
           "gamma": 0.015,
           "loss": "V_CaT_shared",
           "columns_control": ["V", "fluo"],
           "columns_model": ["V", "fluo"],}

In [7]:
dirname_model = "../../models_ctypes/src/model_ctypes/_koivumaki/"
dirname_states = "../../models_ctypes/data/koivumaki/fluo_10uM_ghk/states/"
dirname_phenotypes = "../../models_ctypes/data/koivumaki/fluo_10uM_ghk/phenotypes/"

filename_so = "koivumaki.so"

model_dict = {"filename_so": rel_join(entry_point, dirname_model, filename_so),
              "filename_legend_states": rel_join(entry_point, dirname_model, "legend_states.csv"),
              "filename_legend_constants": rel_join(entry_point, dirname_model, "legend_constants.csv"),
              
              "t_run": 9,
              "t_sampling": 0.001,
              "tol": 1e-4,
              "stim_period_legend_name": "STIM_PERIOD",}

In [8]:
model_dict

{'filename_so': '/home/andrey/WORK/HPL/Code/ga/models_ctypes/src/model_ctypes/_koivumaki/koivumaki.so',
 'filename_legend_states': '/home/andrey/WORK/HPL/Code/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_states.csv',
 'filename_legend_constants': '/home/andrey/WORK/HPL/Code/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_constants.csv',
 't_run': 9,
 't_sampling': 0.001,
 'tol': 0.0001,
 'stim_period_legend_name': 'STIM_PERIOD'}

In [9]:
top_level = {**ga_dict, **model_dict}
top_level["output_folder_name"] = rel_join(entry_point, dirname_results)

## experimental conditions

### common

In [10]:
common = {'params': {}}

common_constants_fixed = {'Mg_i': 1.0,
                          'Ca_o': 2.0,
                          'K_o': 4,
                          'G_seal': 0,
                          #'Na_o': 140,
                          }

common_constants_genes_keys = ['STIM_LEVEL', 'PNa', 'gCaL', 'gt', 'gKur',
                               'gK1', 'gKr', 'gKs', 'INaKmax', 'ICaPmax',
                               'gIf', 'kNaCa', 'Jrel_multiplier', 'cpumps',
                               'kSRleak', 'fluo_tot', 'gNab', 'gCab', 'gKb']

common_constants_genes = {x: {'bounds': [0.1, 10], 'is_multiplier': True} for x in common_constants_genes_keys}

# Custom
common_constants_genes['gNab']['bounds'] = [0.01, 3]
common_constants_genes['gCab']['bounds'] = [0.01, 3]
common_constants_genes['gKb']['bounds'] = [0.01, 3]

common['params'].update(common_constants_fixed)
common['params'].update(common_constants_genes)

### CLs

In [11]:
CL_values = [2000, 1000, 500, 333, 250]

CL_dict = {}

for CL in CL_values:
    CL_dict[CL] = {}
    
    params_fixed = {'STIM_PERIOD': CL / 1000}
    
    params_genes = {'Nai':  {"bounds": [5, 25]    },
                    'Ki':   {"bounds": [100, 300] },
                    'CaSR': {"bounds": [0.08, 8]  }
                   }
    
    CL_dict[CL]['params'] = {**params_fixed, **params_genes}
    
    CL_dict[CL]['filename_phenotype'] = rel_join(entry_point, dirname_phenotypes, f'phenotype_{CL}.csv')
    CL_state = 1000
    CL_dict[CL]['filename_state'] = rel_join(entry_point, dirname_states, f'state_{CL_state}.csv')

In [12]:
config = top_level
config['experimental_conditions'] = {'common': common}
config['experimental_conditions'].update(CL_dict)

In [13]:
with open("../mpi_scripts/voigt/configs/config_test.json", 'w') as f:
    f.write(json.dumps(config, indent=4))